# Chapter 2: Bitcoin Script Fundamentals

> **Reference**: `book/manuscript/Chapter 2 Bitcoin Script Fundamentals - Stack Operations and P2PKH.md`  
> **Code Examples**: `code/chapter02/`  
> **Last Updated**: 2025-12-06

---

Bitcoin's true innovation lies not just in digital signatures or decentralized consensus, but in its programmable money system. Every Bitcoin transaction is essentially a computer program that defines spending conditions. This chapter explores the foundational concepts that make Taproot possible: the UTXO model and Bitcoin Script.

Before diving into scripts, we must understand how Bitcoin represents value.
Unlike traditional banking systems that maintain account balances, Bitcoin uses the **Unspent Transaction Output (UTXO)** model—a system that mimics physical cash more than digital bank accounts.
#
## 

Cash vs.
Banking: A Mental
Mod
e
**Traditional Banking (Account Model)**:
- Your account shows a balance: $500
- Spending $350 simply deducts from your balance
- Result: Account balance updates to $150
- No need to handle "change"
**Bitcoin UTXO Model (Cash Model)**:
- You don't have a "$500 balance"
- Instead, you have specific "bills": one $200 bill and three $100 bills
- To spend $350, you must provide $400 worth of bills ($200 + $100 + $100)
- You receive $50 in change as a new "bill"
- Result: You now have one $100 bill and one $50 bill
This cash-like behavior is fundamental to
Bitcoin's design and security model.
#
## UTXO Model in Practice 

Let's trace a simple transaction between
Alice and B
o
b:
**Initial State**:
- Alice owns a 10 BTC UTXO
- Bob owns no bitcoin
**Alice sends 7 BTC to Bob**:
1.
**Transaction Input**: Alice's 10 BTC UTXO (must be consumed entirely)
2.
**Transaction Outputs**:
- 7 BTC to Bob (new UTXO)
- 3 BTC change back to Alice (new UTXO)
3.
**Result**: The original 10 BTC UTXO is destroyed, two new UTXOs are created
**UTXO Identification**: Each UTXO is uniquely identified by `transaction_id:output_index`
- Bob's UTXO: `TX123:0` (7 BTC)
- Alice's change: `TX123:1` (3 BTC)
#
## 

Key UTXO
Properti
e
s
**Complete Consumption**: UTXOs must be spent in their entirety—no partial spending.
**Atomic Creation**: Transactions either succeed completely (all inputs consumed, all outputs created) or fail completely.
**Change Handling**: Any difference between input and output amounts becomes the transaction fee, unless explicitly returned as change.
**Parallel Processing**: Since each UTXO can only be spent once, multiple transactions can be validated in parallel without complex state management.

#
## Bitcoin 

Script: Programmable Spending Conditions Each UTXO doesn't just contain an amount—it contains a **locking script** (Script Pub Key) that defines the conditions under which it can be spent.
To spend a UTXO, one must provide an **unlocking script** (Script
Sig) that satisfies these conditio
n
s.
#
## 

Script
Architectu
r
e
```
Unlocking Script (Script Sig) + Locking Script (Script Pub Key) → Valid/Invalid
```
**Locking Script (Script Pub Key)**:
- Attached to each UTXO output
- Defines spending conditions
- Example: "Only spendable by someone who can provide a valid signature for public key X"
**Unlocking Script (Script Sig)**:
- Provided when spending a UTXO
- Contains data needed to satisfy the locking script
- Example: "Here's my signature and public key"
**Validation Process**:
- Combine unlocking and locking scripts
- Execute as a single program
- If the final result is TRUE, the UTXO can be spent
#
## Stack-Based Execution Bitcoin Script uses a stack-based execution model, similar to programming languages like Forth or Post 

Script.
Operations manipulate a Last-In-First-Out (LIFO) stack:Initial Stack:
Emp
t
y
```
│ (empty) │└───────────────────────────────────────┘
```
PUSH 3
```
│ 3 │└───────────────────────────────────────┘
```
PUSH 5
```
│ 5 ││ 3 │└───────────────────────────────────────┘
```
ADD Operation
```
│ 8 │└───────────────────────────────────────┘
```
Operation Process:Pop two numbers from stack: 5 (top) and 3Execute addition: 5 + 3 = 8Push result 8 back to stack top This simple model enables complex spending conditions while remaining predictable and secure.
#
## P2PKH: The Foundation Script Pay-to-Public-Key-Hash (P2PKH) is Bitcoin's most fundamental script type and the foundation for understanding more complex scripts like those used in Taproot.**P2PKH Locking
Scri
p
t**
```
OP_DUP OP_HASH160 <pubkey_hash> OP_EQUALVERIFY OP_CHECKSIG
```
This script means: "This UTXO can be spent by anyone who can provide a public key that hashes to `pubkey_hash` and a valid signature from the corresponding private key."**P2PKH Unlocking Script**
```
<signature> <public_key>
```
The spender provides:
- A digital signature proving ownership of the private key
- The public key itself (which will be hashed and verified)
#
## Real-World 

Example: Satoshi to Hal Finney Let's examine the famous first Bitcoin transaction: Satoshi Nakamoto sending 10 BTC to Hal
Finn
e
y.
**Transaction ID**: [`f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16`](https://mempool.space/tx/f4184fc596403b9d638783cf57adfe4c75c605f6356fbc91338530e9831e9e16)
**Transaction Structure**:
-
**Input**: Satoshi's coinbase UTXO (50 BTC from mining)
-
**Outputs**:
- 10 BTC to Hal Finney
- 40 BTC change back to Satoshi
**Note
**: This early transaction used P2PK (Pay-to-Public-Key) rather than P2PKH, directly embedding the public key in the locking script.
Modern Bitcoin uses P2PKH for better security and space efficiency.
#
## Step-by-

Step P2PKH
Executi
o
n
- Hal Finney Example Let's trace through a P2PKH script execution using a real transaction.
Consider Hal Finney later spending his 10 BTC:**
Locking Script** (from the UTXO):
```
OP_DUP OP_HASH160 OP_PUSHBYTES_20 340cfcffe029e6935f4e4e5839a2ff5f29c7a571 OP_EQUALVERIFY OP_CHECKSIG
```
**Unlocking Script** (provided by Hal):
```
OP_PUSHBYTES_71 30440220576497b7e6f9b553c0aba0d8929432550e092db9c130aae37b84b545e7f4a36c022066cb982ed80608372c139d7bb9af335423d5280350fe3e06bd510e695480914f01OP_PUSHBYTES_33 02898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519
```
**Execution Steps**:
1.
**Push Signature to Stack**:
```
│ 30440220...914f01 (signature) │└───────────────────────────────────────┘
```
2.
**Push Public Key to Stack**:
```
│ 02898711...8519 (public_key) ││ 30440220...914f01 (signature) │└───────────────────────────────────────┘
```
3.
**OP_DUP**: Duplicate the top stack item (public key):
```
│ 02898711...8519 (public_key) ││ 02898711...8519 (public_key) ││ 30440220...914f01 (signature) │└───────────────────────────────────────┘
```
4.
**OP_HASH160**: Hash the top stack item:
```
│ 340cfcff...7a571 (hash160_result) ││ 02898711...8519 (public_key) ││ 30440220...914f01 (signature) │└───────────────────────────────────────┘
```
5.
**Push Expected Hash**: From the locking script:
```
│ 340cfcff...7a571 (expected_hash) ││ 340cfcff...7a571 (computed_hash) ││ 02898711...8519 (public_key) ││ 30440220...914f01 (signature) │└───────────────────────────────────────┘
```
6.
**OP_EQUALVERIFY**: Compare top two items, remove both if equal:
```
│ 02898711...8519 (public_key) ││ 30440220...914f01 (signature) │└───────────────────────────────────────┘(Script fails if hashes don't match)
```
7.
**OP_CHECKSIG**: Verify signature against public key and transaction:
```
│ 1 (TRUE) │└───────────────────────────────────────┘
```
8.
**Final Check**: Script succeeds if stack top is non-zero.
#
## P2PKH 

Security
Properti
e
s
**Hash Pre-image Resistance**: The public key remains hidden until first spend, providing protection against potential quantum attacks on ECDSA.
**Signature Verification**: Cryptographically proves the spender controls the private key corresponding to the public key hash.
**Transaction Integrity**: The signature covers the transaction details, preventing modification after signing.
**Replay Protection**: Signatures are specific to particular transactions and cannot be reused.

#
## Crafting a real testnet Legacy-to-Seg 

Wit transaction
Let's build a complete P2PKH transaction step by step, explaining each component and then trace through the script execution with real da
t
a.
#
## Key Functions and 

Components
Explain
e
d
**Setup and Key Management**:
- `setup('testnet')`: Configures the library for testnet operations
- `Private Key()`: Creates a private key object from WIF format
- `P2pkh Address()`: Creates a Legacy address object from address string
- `P2wpkh Address()`: Creates a Seg Wit address object
**Transaction Construction**:
- `Tx Input()`: References a previous UTXO by transaction ID and output index
- `Tx Output()`: Defines where coins are being sent and how much
- `Transaction()`: Combines inputs and outputs into a complete transaction
- `to_satoshis()`: Converts BTC amounts to satoshi units (1 BTC = 100,000,000 satoshis)
**Script and Signature Operations**:
- `to_script_pub_key()`: Generates the locking script for an address
- `sign_input()`: Creates a cryptographic signature for a specific input
- `Script()`: Constructs the unlocking script from signature and public key data
#
## Real Data Analysis and Stack Execution 

Let's analyze the actual data from our transaction execution.
When this code runs, it produces a real transaction that was broadcast to testn
e
t:
**Transaction ID**: [`bf41b47481a9d1c99af0b62bb36bc864182312f39a3e1e06c8f6304ba8e58355`](https://mempool.space/testnet/tx/bf41b47481a9d1c99af0b62bb36bc864182312f39a3e1e06c8f6304ba8e58355)
**Raw Transaction Data**:`0100000001f7061814e7b778978ccb9193559a7832c733655302d2bef3d7f9eca9d0150ab9010000006a473044022055c309fe3f6099f4f881d0fd960923eb91aff0d8ef3501a2fc04dce99aca609d0220174b9aec4fc22f6f81b637bbafec9554e497ec2d9f3ca4992ee4209dd047443d012102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519ffffffff01c8720000000000001600148c7bf2dd2b38b5ed13c3b24ceebf2e7ae30a47df00000000`Let's break down the unlocking script (Script Sig) and trace through its execution:
**Unlocking Script (Script Sig)**:`473044022055c309fe3f6099f4f881d0fd960923eb91aff0d8ef3501a2fc04dce99aca609d0220174b9aec4fc22f6f81b637bbafec9554e497ec2d9f3ca4992ee4209dd047443d012102898711e6bf63f5cbe1b38c05e89d6c391c59e9f8f695da44bf3d20ca674c8519`
**Parsed Components**:
- `47`: OP_PUSHBYTES_71 (push 71 bytes
- the signature)
- `304402...443d01`: DER-encoded signature (71 bytes)
- `21`: OP_PUSHBYTES_33 (push 33 bytes
- the public key)
- `02898711...8519`: Compressed public key (33 bytes)**Locking Script (Script Pub Key)** from the UTXO being spent:`76a914c5b28d6bba91a2693a9b1876bcd3929323890fb288ac`
**Parsed Locking Script**:
- `76`: OP_DUP
- `a9`: OP_HASH160
- `14`: OP_PUSHBYTES_20 (push 20 bytes)
- `c5b28d6bba91a2693a9b1876bcd3929323890fb2`: Public key hash (20 bytes)
- `88`: OP_EQUALVERIFY
- `ac`: OP_CHECKSIG
#
## Stack Execution 

Trace
Now let's trace through the complete script execution step by st
e
p:
**Initial State**:
```
│ (empty) │└───────────────────────────────────────┘
```
Script: <signature> <pubkey> OP_DUP OP_HASH160 <pubkey_hash> OP_EQUALVERIFY OP_CHECKSIG
**Step 1
- Push Signature**:Operation: PUSH 304402...443d01
```
│ 304402...443d01 (signature) │└───────────────────────────────────────┘
```
**Step 2
- Push Public Key**:Operation: PUSH 02898711...8519
```
│ 02898711...8519 (public_key) ││ 304402...443d01 (signature) │└───────────────────────────────────────┘
```
**Step 3
- OP_DUP**:Operation: Duplicate top stack item
```
│ 02898711...8519 (public_key) ││ 02898711...8519 (public_key) ││ 304402...443d01 (signature) │└───────────────────────────────────────┘
```
**Step 4
- OP_HASH160**:Operation: Hash160(top stack item)Calculation: hash160(02898711...8519) = c5b28d6bba91a2693a9b1876bcd3929323890fb2
```
│ c5b28d6b...890fb2 (computed_hash) ││ 02898711...8519 (public_key) ││ 304402...443d01 (signature) │└───────────────────────────────────────┘
```
**Step 5
- Push Expected Hash**:Operation: PUSH c5b28d6bba91a2693a9b1876bcd3929323890fb2
```
│ c5b28d6b...890fb2 (expected_hash) ││ c5b28d6b...890fb2 (computed_hash) ││ 02898711...8519 (public_key) ││ 304402...443d01 (signature) │└───────────────────────────────────────┘
```
**Step 6
- OP_EQUALVERIFY**:Operation: Compare top two items, remove both if equal Verification: c5b28d6b... == c5b28d6b... ✓ (Match!)
```
│ 02898711...8519 (public_key) ││ 304402...443d01 (signature) │└───────────────────────────────────────┘
```
**Step 7
- OP_CHECKSIG**:Operation: Verify signature against public key and transaction Inputs:Public key: 02898711...8519Signature: 304402...443d01Transaction data: (serialized transaction for signature)Verification: ECDSA verification ✓ (Valid signature!)
```
│ 1 (TRUE) │└───────────────────────────────────────┘
```
**Final State**:
```
│ 1 (TRUE) │└───────────────────────────────────────┘
```
Result: SUCCE
S
S (non-zero value on stack)
#
## Transaction Broadcast Result 

This transaction was successfully broadcast to the
Bitcoin testnet and can be viewed at:https://mempool.space/testnet/tx/bf41b47481a9d1c99af0b62bb36bc864182312f39a3e1e06c8f6304b
a8
e58355
**Key Observations**:
- The input references UTXO from transaction `34b90a15...1806f7` at index 1
- The output sends 29,400 satoshis to a Seg Wit address
- The transaction fee is 206 satoshis (29,606
- 29,400)
- The signature verification proves ownership of the private key without revealing it
#
## From P2PKH to Advanced ScriptsP2PKH provides the foundation for understanding 

Bitcoin's programmable money system, but it's just the beginning.
The same principles—stack-based execution, cryptographic verification, and conditional logic—enable more sophisticated scripts that we'll explore in upcoming chapte
r
s:
**P2SH (Pay-to-Script-Hash)**:
- Enables complex spending conditions while keeping addresses short
- Moves script complexity from the blockchain to the spender
- Foundation for wrapped Seg Wit and multi-signature schemes
**P2WPKH (Pay-to-Witness-Public-Key-Hash)**:
- Seg Wit's equivalent to P2PKH with improved efficiency
- Separates signature data from transaction data
- Reduces transaction malleability and enables Lightning Network
**P2TR (Pay-to-Taproot)**:
- The culmination of Bitcoin's script evolution
- Enables complex smart contracts that look like simple payments
- Combines Schnorr signatures with Merkle trees for maximum flexibility Each evolution maintains backward compatibility while adding new capabilities.
Understanding P2PKH's stack execution model is crucial because Taproot uses the same fundamental approach, just with more sophisticated cryptographic primitives and script structures.
In the next chapter, we'll dive deep into these address types, examining their script structures and understanding how each improvement builds upon the lessons learned from P2PKH.

This chapter established the foundational concepts that make Taproot possible:
**UTXO Model**: Bitcoin represents value as discrete, spendable outputs rather than account balances.
Each UTXO must be consumed entirely, creating a cash-like system that enables parallel transaction validation and eliminates complex state management.
**Script System**: Each UTXO contains programmable spending conditions through locking scripts (Script Pub Key).
To spend a UTXO, one must provide an unlocking script (Script Sig) that satisfies these conditions when executed together.
**Stack Execution**: Bitcoin Script uses a simple stack-based model for processing conditions, where operations manipulate a Last-In-First-Out stack to verify spending authorization.
**P2PKH Implementation**: The fundamental script type demonstrates signature verification and public key validation through a seven-step process: signature and public key provision, duplication, hashing, comparison, and signature verification.
**Practical Development**: Using tools like [`bitcoinutils`](https://github.com/karask/python-bitcoin-utils), developers can construct, sign, and broadcast P2PKH transactions while understanding the underlying cryptographic operations and stack execution.
Understanding these concepts is crucial because Taproot builds upon them, using the same stack-based execution model while introducing new cryptographic primitives and script structures.
The journey from simple P2PKH scripts to Taproot's sophisticated spending conditions illustrates Bitcoin's evolution from basic digital cash to a platform for complex financial applications—all while preserving the security and simplicity that make Bitcoin unique.